In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import mnist
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

2025-05-22 08:53:42.586108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747904022.782531      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747904022.842414      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# === Load MNIST dataset ===
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.
x_train = np.expand_dims(x_train, axis=-1)  # (60000, 28, 28, 1)
x_test = np.expand_dims(x_test, axis=-1)    # (10000, 28, 28, 1)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
# === Create CNN feature extractor ===
def create_cnn_model(conv1_filters=6, conv2_filters=12, kernel_size=(5,5),
                     dense_units=100, dropout_rate=0.5, hidden_units=64, activation='relu'):
    input_img = Input(shape=(28, 28, 1))
    x = Conv2D(conv1_filters, kernel_size, activation=activation)(input_img)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(conv2_filters, kernel_size, activation=activation)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(dense_units, activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    feature_output = Dense(hidden_units, activation=activation)(x)
    model = Model(inputs=input_img, outputs=feature_output)
    return model

In [ ]:
# hyperparameter grid for CNN and SVM
from itertools import product

cnn_param_grid = {
    'conv1_filters': [8, 16],
    'conv2_filters': [16, 32],
    'kernel_size': [(3, 3), (5, 5)],
    'dense_units': [100],
    'dropout_rate': [0.5],
    'activation': ['relu'],
    'hidden_units': [64],
}
# 
svm_param_grid = {
    'C': [0.5, 1, 10],
    'gamma': ['scale', 0.1, 1]
}


In [6]:
# === Keep best model and config ===
best_score = 0
best_model = None
best_config = None
best_features_test = None

# === Generate all combinations of CNN hyperparameters ===
cnn_keys = list(cnn_param_grid.keys())
cnn_values = list(cnn_param_grid.values())
cnn_param_combinations = list(product(*cnn_values))

print(f"[INFO] Total CNN configurations to try: {len(cnn_param_combinations)}")

for cnn_params in cnn_param_combinations:
    config = dict(zip(cnn_keys, cnn_params))
    print(f"\n[INFO] Training CNN with config: {config}")

    # Build and run CNN
    cnn = create_cnn_model(**config)
    features_train = cnn.predict(x_train, batch_size=64, verbose=0)
    features_test = cnn.predict(x_test, batch_size=64, verbose=0)

    # Normalize
    scaler = StandardScaler()
    features_train_scaled = scaler.fit_transform(features_train)
    features_test_scaled = scaler.transform(features_test)

    # Train SVM with Grid Search
    print(f"[INFO] Performing SVM grid search (RBF Kernel)")
    svm = SVC(kernel='rbf')
    clf = GridSearchCV(svm, svm_param_grid, cv=4, verbose=0, n_jobs=-1)
    clf.fit(features_train_scaled, y_train)

    acc = clf.score(features_test_scaled, y_test)
    print(f"[INFO] CNN Config = {config}, SVM Best Params = {clf.best_params_}, Test Accuracy = {acc:.4f}")

    # Keep best model
    if acc > best_score:
        best_score = acc
        best_model = clf
        best_config = {
            'cnn_params': config,
            'svm_params': clf.best_params_
        }
        best_features_test = features_test_scaled



[INFO] Total CNN configurations to try: 8

[INFO] Training CNN with config: {'conv1_filters': 8, 'conv2_filters': 16, 'kernel_size': (3, 3), 'dense_units': 100, 'dropout_rate': 0.5, 'activation': 'relu', 'hidden_units': 64}


I0000 00:00:1747904036.667956      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1747904036.668664      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
I0000 00:00:1747904038.509483      62 service.cc:148] XLA service 0x791e2c004720 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747904038.510233      62 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1747904038.510251      62 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1747904038.598822      62 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1747904039.640994      62 device_compiler.h:188] Compiled clust

[INFO] Performing SVM grid search (RBF Kernel)
[INFO] CNN Config = {'conv1_filters': 8, 'conv2_filters': 16, 'kernel_size': (3, 3), 'dense_units': 100, 'dropout_rate': 0.5, 'activation': 'relu', 'hidden_units': 64}, SVM Best Params = {'C': 10, 'gamma': 'scale'}, Test Accuracy = 0.9456

[INFO] Training CNN with config: {'conv1_filters': 8, 'conv2_filters': 16, 'kernel_size': (5, 5), 'dense_units': 100, 'dropout_rate': 0.5, 'activation': 'relu', 'hidden_units': 64}
[INFO] Performing SVM grid search (RBF Kernel)
[INFO] CNN Config = {'conv1_filters': 8, 'conv2_filters': 16, 'kernel_size': (5, 5), 'dense_units': 100, 'dropout_rate': 0.5, 'activation': 'relu', 'hidden_units': 64}, SVM Best Params = {'C': 10, 'gamma': 'scale'}, Test Accuracy = 0.9305

[INFO] Training CNN with config: {'conv1_filters': 8, 'conv2_filters': 32, 'kernel_size': (3, 3), 'dense_units': 100, 'dropout_rate': 0.5, 'activation': 'relu', 'hidden_units': 64}
[INFO] Performing SVM grid search (RBF Kernel)
[INFO] CNN Config

In [7]:
# === Final Evaluation ===
print("\n===== FINAL REPORT =====")
print(f"\n=== Best Accuracy: {best_score:.4f} ===")
print(f"Best CNN Config: {best_config['cnn_params']}")
print(f"Best SVM Config: {best_config['svm_params']}")

y_pred = best_model.predict(best_features_test)
print("Classification Report on MNIST Test Set:")
print(classification_report(y_test, y_pred))



===== FINAL REPORT =====

=== Best Accuracy: 0.9456 ===
Best CNN Config: {'conv1_filters': 8, 'conv2_filters': 16, 'kernel_size': (3, 3), 'dense_units': 100, 'dropout_rate': 0.5, 'activation': 'relu', 'hidden_units': 64}
Best SVM Config: {'C': 10, 'gamma': 'scale'}
Classification Report on MNIST Test Set:
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       980
           1       0.98      0.99      0.99      1135
           2       0.95      0.95      0.95      1032
           3       0.93      0.93      0.93      1010
           4       0.92      0.92      0.92       982
           5       0.93      0.93      0.93       892
           6       0.97      0.96      0.96       958
           7       0.95      0.93      0.94      1028
           8       0.93      0.92      0.92       974
           9       0.91      0.93      0.92      1009

    accuracy                           0.95     10000
   macro avg       0.95      0.94      0.9

# Result